In [25]:
import torch
from torchvision import models
import torch.nn as nn

# Create ResNet18 with the correct number of output classes (2) to match the checkpoint
model = models.resnet18(num_classes=2)

# load with map_location and allow missing/unexpected keys to avoid RuntimeError
checkpoint = torch.load("pytorch_model.bin", map_location=torch.device('cpu'))
model.load_state_dict(checkpoint, strict=False)

# Move model to available device and set to eval mode
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
# model.eval()

In [26]:
from torchvision import transforms

preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [13]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("cashbowman/ai-generated-images-vs-real-images")

print("Path to dataset files:", path)

100%|██████████| 476M/476M [00:43<00:00, 11.4MB/s] 

Extracting files...


Path to dataset files: /home/luis-menezes/.cache/kagglehub/datasets/cashbowman/ai-generated-images-vs-real-images/versions/1


In [27]:
import zipfile
from pathlib import Path
from PIL import Image
import torch
import torch.nn.functional as F
from collections import Counter

# ...existing code...

# Evaluate model on images found under `path`
data_path = Path(path)  # `path` comes from your kagglehub download cell

# If dataset is a zip, extract it next to the zip
if data_path.is_file() and data_path.suffix == ".zip":
    extract_dir = data_path.with_suffix("")
    if not extract_dir.exists():
        with zipfile.ZipFile(data_path, "r") as z:
            z.extractall(extract_dir)
    data_path = extract_dir

if not data_path.exists():
    raise FileNotFoundError(f"Dataset path not found: {data_path}")

# Find image files and possible class folders
IMG_EXTS = {".jpg", ".jpeg", ".png", ".bmp"}
subdirs = [p for p in data_path.iterdir() if p.is_dir()]
if subdirs:
    classes = sorted([d.name for d in subdirs])
    image_files = [p for d in subdirs for p in d.rglob("*") if p.suffix.lower() in IMG_EXTS]
else:
    classes = []
    image_files = [p for p in data_path.rglob("*") if p.suffix.lower() in IMG_EXTS]

if not image_files:
    print("No images found under", data_path)
else:
    device = next(model.parameters()).device if any(True for _ in model.parameters()) else torch.device("cpu")
    totals = 0
    correct = 0
    per_class = Counter()
    per_class_tot = Counter()

    for img_path in image_files:
        img = Image.open(img_path).convert("RGB")
        tensor = preprocess(img).unsqueeze(0).to(device)

        with torch.no_grad():
            out = model(tensor)

            # Binary output (single logit)
            if out.shape[1] == 1:
                prob_pos = torch.sigmoid(out).item()
                pred_idx = 1 if prob_pos > 0.5 else 0
                pred_label = classes[pred_idx] if len(classes) >= 2 else str(pred_idx)
                conf = prob_pos if pred_idx == 1 else 1.0 - prob_pos

            # Multi-class and number of outputs matches folder classes
            elif len(classes) > 0 and out.shape[1] == len(classes):
                probs = F.softmax(out, dim=1)
                conf, idx = probs.max(dim=1)
                pred_idx = int(idx.item())
                pred_label = classes[pred_idx]
                conf = float(conf.item())

            # Otherwise fallback to imagenet-like labels_file
            else:
                probs = F.softmax(out, dim=1)
                conf, idx = probs.max(dim=1)
                pred_idx = int(idx.item())
                pred_label = labels_file[pred_idx] if pred_idx < len(labels_file) else str(pred_idx)
                conf = float(conf.item())

        # True label from parent folder if available
        true_label = img_path.parent.name if img_path.parent != data_path else None

        print(f"{img_path} | true: {true_label} | pred: {pred_label} (conf {conf:.3f})")

        if true_label is not None:
            totals += 1
            per_class_tot[true_label] += 1
            if true_label == pred_label:
                correct += 1
                per_class[true_label] += 1

    if totals > 0:
        acc = correct / totals
        print(f"\nOverall accuracy (using parent-folder labels): {acc:.4f} ({correct}/{totals})")
        print("Per-class accuracy:")
        for c in sorted(per_class_tot):
            print(f"  {c}: {per_class[c]}/{per_class_tot[c]} = {per_class[c]/per_class_tot[c]:.4f}")
    else:
        print("\nNo parent-folder labels found -> accuracy not computed. Only predictions printed.")

/home/luis-menezes/.cache/kagglehub/datasets/cashbowman/ai-generated-images-vs-real-images/versions/1/AiArtData/AiArtData/a4411c44-c1cc-45ca-bf4b-2ecaf5172ba8.jpg | true: AiArtData | pred: RealArt (conf 0.942)
/home/luis-menezes/.cache/kagglehub/datasets/cashbowman/ai-generated-images-vs-real-images/versions/1/AiArtData/AiArtData/ai-generated-photos-aurel-manea-1.jpg | true: AiArtData | pred: RealArt (conf 0.938)
/home/luis-menezes/.cache/kagglehub/datasets/cashbowman/ai-generated-images-vs-real-images/versions/1/AiArtData/AiArtData/images12.jpg | true: AiArtData | pred: RealArt (conf 0.944)
/home/luis-menezes/.cache/kagglehub/datasets/cashbowman/ai-generated-images-vs-real-images/versions/1/AiArtData/AiArtData/230726115314-01-lensa-selfie.jpg | true: AiArtData | pred: RealArt (conf 0.957)
/home/luis-menezes/.cache/kagglehub/datasets/cashbowman/ai-generated-images-vs-real-images/versions/1/AiArtData/AiArtData/nvidia_ai.png | true: AiArtData | pred: RealArt (conf 0.941)
/home/luis-menez

/home/luis-menezes/miniconda3/envs/env_3.12/lib/python3.12/site-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


/home/luis-menezes/.cache/kagglehub/datasets/cashbowman/ai-generated-images-vs-real-images/versions/1/AiArtData/AiArtData/ai-portraits-ars.jpg | true: AiArtData | pred: RealArt (conf 0.925)
/home/luis-menezes/.cache/kagglehub/datasets/cashbowman/ai-generated-images-vs-real-images/versions/1/AiArtData/AiArtData/il_570xN.4585214944_j5ob.jpg | true: AiArtData | pred: RealArt (conf 0.950)
/home/luis-menezes/.cache/kagglehub/datasets/cashbowman/ai-generated-images-vs-real-images/versions/1/AiArtData/AiArtData/Boris-Eldagsen-AI-art.jpg | true: AiArtData | pred: RealArt (conf 0.902)
/home/luis-menezes/.cache/kagglehub/datasets/cashbowman/ai-generated-images-vs-real-images/versions/1/AiArtData/AiArtData/nGuYVYiPu3dnmigzNfdMLZ-1200-80.jpg | true: AiArtData | pred: RealArt (conf 0.932)
/home/luis-menezes/.cache/kagglehub/datasets/cashbowman/ai-generated-images-vs-real-images/versions/1/AiArtData/AiArtData/ai-generated-8146218_1280.png | true: AiArtData | pred: RealArt (conf 0.940)
/home/luis-men

In [11]:
from matplotlib import pyplot as plt
import numpy as np
import requests
try:
    labels_url = "https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt"
    labels_file = requests.get(labels_url).text.split("\n")
except Exception as e:
    print(f"Erro ao baixar labels: {e}")
    # Fallback se a internet falhar na apresentação
    labels_file = ["Label N/A"] * 1000

# Função para prever (classificar) a imagem
def predict(image_tensor, top_k=1):
    with torch.no_grad(): # Não calcular gradientes na inferência normal
        output = model(image_tensor)
        probabilities = F.softmax(output, dim=1)
        top_prob, top_indices = torch.topk(probabilities, top_k)
        
        for i in range(top_k):
            label = labels_file[top_indices[0][i]]
            prob = top_prob[0][i].item() * 100
            print(f"Predição {i+1}: {label} (Confiança: {prob:.2f}%)")
    return top_indices[0][0] # Retorna o índice da classe principal

# Função para mostrar a imagem (desnormalizando)
def show_image(tensor, title=""):
    # Desnormalizar a imagem para visualização
    tensor = tensor.clone().detach().cpu().squeeze(0)
    inv_normalize = transforms.Normalize(
        mean=[-0.485/0.229, -0.456/0.224, -0.406/0.225],
        std=[1/0.229, 1/0.224, 1/0.225]
    )
    img = inv_normalize(tensor)
    img = img.permute(1, 2, 0) # Mudar de [C, H, W] para [H, W, C]
    img = np.clip(img, 0, 1) # Garantir que os valores estão entre 0 e 1
    plt.imshow(img)
    plt.title(title)
    plt.axis('off')
    plt.show()